In [ ]:
from pathlib import Path
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, to_json, col, udf, explode, lit, coalesce, sha2
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

In [ ]:
spark = (
                SparkSession
                .builder
                .appName("fmcg-data-analysis")
                .master("local")
                .config("spark.sql.repl.eagerEval.enabled", True)
                .getOrCreate()
    )

In [ ]:
curr_path = Path().resolve()
pub_path = Path(str(curr_path).replace('/test', '/publish')).resolve()
out_path = str(Path(pub_path) / 'psuedo_fmcg')
print(out_path)

In [ ]:
df = spark.read.parquet('/Users/keerthan/Projects/FMCG_Analysis/publish/fmcg_analysis')
df.count()
df.createOrReplaceTempView('df')
df.printSchema()

In [ ]:
psuedo_df = df.withColumn('address', col('address').withField('psuedo_houseNumber', sha2(col('address.houseNumber'), 256)))\
    .withColumn('address', col('address').withField('psuedo_streetName', sha2(col('address.streetName'), 256)))

In [ ]:
psuedo_df.write.partitionBy('brand').mode('overwrite').parquet(out_path)

In [ ]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()

def generate_encrypt_key():
    from cryptography.fernet import Fernet
    key = Fernet.generate_key()
    return key.decode("utf-8")

#spark.udf.register("generate_key_using_Fernet", generate_encrypt_key)


In [ ]:
generate_key_using_Fernet = udf(generate_encrypt_key, StringType())
df_distinct_record = spark.sql('''select distinct placeId from df''')
df_distinct_record.count()
df_distinct_record = df_distinct_record.withColumn("encryption_key", lit(generate_key_using_Fernet()))
df_distinct_record.select('placeId', 'encryption_key').show(10,0)
df_distinct_record.write.option('header', 'True').mode('overwrite').csv('/Users/keerthan/Projects/FMCG_Analysis/publish/encryption_keys')

In [ ]:
# Define Encrypt User Defined Function 
def encrypt_val(clear_text,MASTER_KEY):
    from cryptography.fernet import Fernet
    f = Fernet(MASTER_KEY)
    clear_text_b=bytes(clear_text, 'utf-8')
    cipher_text = f.encrypt(clear_text_b)
    cipher_text = str(cipher_text.decode('ascii'))
    return cipher_text

# Define decrypt user defined function 
def decrypt_val(cipher_text,MASTER_KEY):
    from cryptography.fernet import Fernet
    f = Fernet(MASTER_KEY)
    clear_val=f.decrypt(cipher_text.encode()).decode()
    return clear_val
#spark.udf.register("decrypt_val", decrypt_val)

In [ ]:
encrypr_df = spark.read.option('InferSchema', 'True')\
    .option('header', 'True')\
    .csv('/Users/keerthan/Projects/FMCG_Analysis/publish/encryption_keys')
encrypr_df.show(2,0)
encrypr_df.createOrReplaceTempView('encryption_keys')
encrypr_df.printSchema()

In [ ]:
# Register UDF's
encrypt = udf(encrypt_val, StringType())
decrypt = udf(decrypt_val, StringType())
 
 
# Encrypt the data 
place_enc_join_df = spark.sql('''select a.*, e.encryption_key 
                                    from df as a 
                                inner join encryption_keys as e 
                                on e.placeId = a.placeId''')
encrypted = df.withColumn("EMAIL", encrypt("EMAIL", col("encryption_Key"))).drop("encryption_Key")

 
#Save encrypted data 
encrypted.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("gdpr.raw_customer_data")  

In [ ]:
encrypt = udf(encrypt_val, StringType())
decrypt = udf(decrypt_val, StringType())

In [45]:
# Encrypt the data 
place_enck_join_df = spark.sql('''select a.*, e.encryption_key 
                                    from df as a 
                                inner join encryption_keys as e 
                                on e.placeId = a.placeId''')

masked_df = place_enck_join_df.withColumn('address', col('address').withField('houseNumber', encrypt("address.houseNumber", col("encryption_Key"))))\
                 .withColumn('address', col('address').withField('streetName', encrypt("address.streetName", col("encryption_Key"))))\
                 .drop("encryption_Key")

masked_df.createOrReplaceTempView('masked_df')
#masked_df.printSchema()
masked_df.select('placeId', 'address').show(2,0)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|placeId|address                                                                                                                                                                                                                              |
+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|3577   |{AALST, BE, België, gAAAAABmDa0n2m4e2THydZDb1pbZT2Y6gRhgMeCK9xnLoIo0Ajc3Y8rcXZIPkIZvhHWpcLfIXReb58gLr4zIRIuft-Pcv077zw==, 9300, gAAAAABmDa0n-Ax9zspOrea-JXwvEHoDSLnJpGnCzqz7tCijSP0jGqerHd7Cep6Xt6bWZw_DlVww1zvB823Wy3rwcMrjQ1ZQ_A==}|
|3577   |{AALST, BE, België, gAAAAABmDa0

In [ ]:
spark.sql(" select placeId, count(*) from df group by placeId having count(*) <= 1").show(10,0)
where a.placeId = 2453

In [46]:
#Decrypt

encrypted = spark.sql('''select a.*, e.encryption_key 
                                    from masked_df as a 
                                inner join encryption_keys as e 
                                on e.placeId = a.placeId
                      ''')


In [48]:
unmasked_df = encrypted.withColumn('address', col('address').withField('houseNumber', decrypt("address.houseNumber", col("encryption_Key"))))\
    .withColumn('address', col('address').withField('streetName', decrypt("address.streetName", col("encryption_Key"))))\
    .drop("encryption_Key")

unmasked_df.select('placeId', 'address').show(2,0)


+-------+-----------------------------------------------+
|placeId|address                                        |
+-------+-----------------------------------------------+
|3577   |{AALST, BE, België, 369, 9300, GENTSE STEENWEG}|
|3577   |{AALST, BE, België, 369, 9300, GENTSE STEENWEG}|
+-------+-----------------------------------------------+
only showing top 2 rows



In [ ]:
encrypted = spark.sql('''select a.*,e.encryption_key from gdpr.raw_customer_data as a 
inner join gdpr.encryption_keys as e on e.ID=a.ID''')

decrypted = encrypted.withColumn("EMAIL", decrypt("EMAIL",(col("encryption_Key")))).drop("encryption_Key")
decrypted.select("ID", "EMAIL","customer_pseudo_id" ).show(2,0)